In [7]:
#for audio processing
from Timecode import Timecode
from pydub import AudioSegment

#for create process purpose
import subprocess

#for reading the file
import os
import shutil
import glob
import simplejson as json
audio_path = 'UclassAudioV1'
script_path = 'GentleAligned20201201'
output_path = 'SplittedAudio'

In [3]:
class Segment:
    def __init__(self,segment_id=0,file_name='Default.mp4',start_time=Timecode(),
                end_time=Timecode(), label='None'):
        self.segmentID = segment_id
        self.fileName = file_name
        self.startTime = start_time
        self.endTime = end_time
        self.label = label
        
    def get_info(self):
        print("Segment ID : " + str(self.segmentID))
        print("File Name  : " + str(self.fileName))
        print("Start Time : " + str(self.startTime.get_timecode()))
        print("End Time   : " + str(self.endTime.get_timecode()))
        print("Label      : " + str(self.label))
    
    def export_audio(self):
        print("Exporting audio")

In [4]:
class Gentle:
    def __init__(self,json_data = "none",verbose=True):
        self.jsonData = json_data
        self.List_of_audio = []
        self.verbose=verbose

    def load_json(self,file_name):
        if os.path.exists(file_name):
            file = open(file_name)
            self.jsonData = json.load(file)
            file.close
        else:
            print(file_name + "does not exist!")
    
    def create_subdir(self,output_path='SplittedAudio'):
        self.output_path = output_path
        subDirs = ['success','not-found-in-transcript','stuttered']
        for subDir in subDirs:
            subDir = self.output_path + '/' + subDir
#             if os.path.exists(f'{subDir}'):
#                 shutil.rmtree(f'{subDir}')
#                 print("Removed " + f'{subDir}')
            if not os.path.exists(f'{subDir}'):
                os.mkdir(f'{subDir}')
                print("Created " + f'{subDir}')
#             os.mkdir(f'{subDir}')
#             print("Created " + f'{subDir}')        
        print("Gentle created!")
    
    def split_audio(self,audio_path='example.wav',json_path='example.json',
                output_path='output.wav',prefix=''):
        self.json_path = json_path
        self.audio_path = audio_path
        self.output_path = output_path
        self.prefix = prefix
        if not os.path.exists(self.audio_path):
            print(self.audio_path+' not exist!')
            return
        if not os.path.exists(self.json_path):
            print(self.json_path+' not exist!')
            return
        if os.path.exists(self.output_path):
            print(self.output_path+' exist')
#             return
        else:
            os.mkdir(self.output_path)
            print('Created ' + output_path)
        self.create_subdir(self.output_path);
        self.load_json(self.json_path)
        words = self.jsonData['words']
        
        song = AudioSegment.from_wav(self.audio_path)
        
        my_list = []
        not_found_in_audio = []
        for i in words:
            case = i["case"]
            if case == "success":
                alignedWord = i["alignedWord"]
                startTime = i["start"]*1000
                endTime = i["end"]*1000
                my_list.append({ "alignedWord" : alignedWord, "startTime" : startTime, "endTime": endTime, "case": case})
            elif(case == "not-found-in-transcript"):
                startTime = i["start"]*1000
                endTime = i["end"]*1000
                my_list.append({ "startTime" : startTime, "endTime": endTime, "case": case})
            else:
                not_found_in_audio.append(i["word"])
                
        List_of_audio = []
        for i in range(len(my_list)):
            index = i
            case = my_list[index]['case']
            startTime = float(my_list[index]['startTime'])
            endTime = float(my_list[index]['endTime'])
            id = '{:06d}'.format(i)
            duration = endTime - startTime
            if duration < 100:
                print('skiped')
                continue
            if case == 'success':
                alignedWord = my_list[index]['alignedWord']
                alignedWord = [character for character in alignedWord if character.isalnum()]
                alignedWord = "".join(alignedWord)
                if (alignedWord == 'er' or alignedWord == 'ah' or alignedWord == 'um' or alignedWord == 'uh' or alignedWord == 'unk' and duration > 100):
                    List_of_audio.append([song[startTime:endTime],f'{self.output_path}/stuttered/{prefix}{id}-{alignedWord}.wav'])
                else:
                    List_of_audio.append([song[startTime:endTime],f'{self.output_path}/success/{prefix}{id}-{alignedWord}.wav'])
                if self.verbose:
                    print("id=",id, " case=",case," startTime=", startTime," endTime=", endTime, 
                      " alignedWord=",alignedWord, "duration=",duration)
            elif case == 'not-found-in-transcript':
                if duration < 100:
                    print("skiped ",id)
                    continue
                print("id=", id," case=",case," startTime=", startTime," endTime=", endTime,"duration=",duration)
                List_of_audio.append([song[startTime:endTime],f'./not-found-in-transcript/{id}.wav'])        
        
        if self.verbose:
            for audio in List_of_audio:
                print(audio)
        for i in List_of_audio:
            i[0].export(i[1],format='wav')
            print(f'exported {i[1]}')

In [8]:
files = glob.glob(f'{script_path}/*.json')
files_name = []
for file in files:
    print(file.split('\\')[1].split('.')[0])
    files_name.append(file.split('\\')[1].split('.')[0])
    
for file_name in files_name:
    a = Gentle(verbose=False).split_audio(f'UclassAudioV1/{file_name}.wav',
                             f'GentleAligned20201201/{file_name}.json',f'20201201-all',
                             prefix=file_name+'-')

M_0100_12y3m_1
M_1017_11y8m_1
20201201-all exist
Created 20201201-all/success
Created 20201201-all/not-found-in-transcript
Created 20201201-all/stuttered
Gentle created!
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
skiped
exported 20201201-all/success/M_0100_12y3m_1-000000-work.wav
exported 20201201-all/success/M_0100_12y3m_1-000001-this.wav
exported 20201201-all/success/M_0100_12y3m_1-000004-end.wav
exported 20201201-all/success/M_0100_12y3m_1-000008-east.wav
exported 20201201-all/success/M_0100_12y3m_1-000010-the.wav
exported 20201201-all/success/M_0100_12y3m_1-000012-and.wav
exported 20201201-all/success/M_0100_12y3m_1-000013-buffy.wav
exported 20201201-all/succe

In [ ]:
# a = Gentle(verbose=False).split_audio('UclassAudioV1/M_0078_16y5m_1.wav',
#                          'GentleAlignedBeautify/M_0078_16y5m_1.json','uniclass',
#                          prefix='M_0078_16y5m_1-')

All below is prototype!

In [ ]:
# json_path = 'GentleAlignedBeautify/F_0101_10y4m_1.json'
# audio_path = 'UclassAudioV1/F_0101_10y4m_1.wav'
# prefix = ''
# file = open(json_path)
# jsonData = json.load(file)
# song = AudioSegment.from_wav(audio_path)
# file.close()
# words = jsonData['words']

In [ ]:
# my_list = []
# not_found_in_audio = []
# for i in words:
# 	case = i["case"]
# 	if case == "success":
# 		alignedWord = i["alignedWord"]
# 		startTime = i["start"]*1000
# 		endTime = i["end"]*1000
# 		my_list.append({ "alignedWord" : alignedWord, "startTime" : startTime, "endTime": endTime, "case": case})
# 	elif(case == "not-found-in-transcript"):
# 		startTime = i["start"]*1000
# 		endTime = i["end"]*1000
# 		my_list.append({ "startTime" : startTime, "endTime": endTime, "case": case})
# 	else:
# 		not_found_in_audio.append(i["word"])

In [ ]:
# List_of_audio = []

In [ ]:
# for i in range(len(my_list)):
#     index = i
#     case = my_list[index]['case']
#     startTime = float(my_list[index]['startTime'])
#     endTime = float(my_list[index]['endTime'])
#     id = '{:06d}'.format(i)
#     duration = endTime - startTime
#     if case == 'success':
#         alignedWord = my_list[index]['alignedWord']
#         alignedWord = [character for character in alignedWord if character.isalnum()]
#         alignedWord = "".join(alignedWord)
#         if (alignedWord == 'um' or alignedWord == 'uh' or alignedWord == 'unk' and duration > 100):
#             List_of_audio.append([song[startTime:endTime],f'./stuttered/{prefix}{id}-{alignedWord}.wav'])
#         else:
#             List_of_audio.append([song[startTime:endTime],f'./success/{prefix}{id}-{alignedWord}.wav'])
#         print("id=",id, " case=",case," startTime=", startTime," endTime=", endTime, 
#               " alignedWord=",alignedWord, "duration=",duration)
#     elif case == 'not-found-in-transcript':
#         if duration < 100:
#             print("skiped ",id)
#             continue
#         print("id=", id," case=",case," startTime=", startTime," endTime=", endTime,"duration=",duration)
#         List_of_audio.append([song[startTime:endTime],f'./not-found-in-transcript/{id}.wav'])

In [ ]:
# for audio in List_of_audio:
#     print(audio)